In [ ]:
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(pheatmap))
suppressPackageStartupMessages(library(stringr))
library(hise)
library(plyr)
library(purrr)
library(vegan)
#library(arrow)
library(parallel)
library(ComplexHeatmap)
library(ggsci)
library(tidyverse)

# Creating merged df with metadata and pseudobulk mean expression

In [50]:
mean_df<-read.csv("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/CertPro_analysis/scripts/Figure_Notebooks/Intermediate_files/Mean_gene_expression/IL1B_CD14mono_pseudobulk_genes_expMatrix.csv", row.names=1)
dim(mean_df)

[1] 32 92

In [51]:
meta_data=read.csv("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/CertPro_analysis/scripts/scRNA_meta_data-2024-05-09.csv")
colnames(meta_data)

[1] "X"                                        
 [2] "Unnamed..0"                               
 [3] "lastUpdated"                              
 [4] "sample.id"                                
 [5] "sample.bridgingControl"                   
 [6] "sample.sampleKitGuid"                     
 [7] "sample.visitName"                         
 [8] "sample.visitDetails"                      
 [9] "sample.drawDate"                          
[10] "sample.daysSinceFirstVisit"               
[11] "file.id"                                  
[12] "file.name"                                
[13] "file.batchID"                             
[14] "file.panel"                               
[15] "file.pool"                                
[16] "file.fileType"                            
[17] "file.majorVersion"                        
[18] "subject.id"                               
[19] "subject.biologicalSex"                    
[20] "subject.birthYear"                        
[21] "subject.ethnicity"                        
[22] "subject.partnerCode"                      
[23] "subject.race"                             
[24] "subject.subjectGuid"                      
[25] "cohort.cohortGuid"                        
[26] "sample.diseaseStatesRecordedAtVisit"      
[27] "pbmc_sample_id"                           
[28] "subject.covidVaxDose1.daysSinceFirstVisit"
[29] "subject.covidVaxDose2.daysSinceFirstVisit"
[30] "Covid_exclusion"                          
[31] "subjectGuid"                              
[32] "CMV"

In [52]:
unique(meta_data$sample.visitDetails)
unique(meta_data$sample.visitName)
unique(meta_data$CMV)
table(meta_data$CMV)


[1] "N/A - Flu-Series Timepoint Only" "Immune Variation Day 0"         
[3] "Immune Variation Day 7"          "Immune Variation Day 90"

[1] "Flu Year 1 Day 0"        "Flu Year 1 Day 7"       
 [3] "Flu Year 1 Day 90"       "Immune Variation Day 0" 
 [5] "Immune Variation Day 7"  "Immune Variation Day 90"
 [7] "Flu Year 2 Stand-Alone"  "Flu Year 1 Stand-Alone" 
 [9] "Flu Year 2 Day 0"        "Flu Year 2 Day 7"       
[11] "Flu Year 2 Day 90"       "Flu Year 3 Stand-Alone"

[1] "Negative" "Positive"


Negative Positive 
     476      392 

In [53]:
# meta_data=read.csv('/home/jupyter/IH-A-Aging-Analysis-Notebooks/Qiuyu-Notebooks/scRNA/hise_meta_data_2024-01-23_fixed.csv')
# colnames(meta_data)
# #unique(meta_data$pbmc_sample_id)
# #length(unique(meta_data$pbmc_sample_id))

In [54]:
dim(mean_df)

[1] 32 92

In [55]:
head(mean_df, n=2)

,PB00001.01,PB00002.01,PB00003.01,PB00004.01,PB00006.01,PB00010.02,PB00012.01,PB00014.01,PB00015.01,PB00016.01,⋯,PB00592.01,PB00599.01,PB00600.01,PB00602.01,PB00621.01,PB00623.01,PB00625.02,PB00645.01,PB01425.01,PB01446.01
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PTGS2,0.4266934,0.2266424,0.06349906,0.7218853,0.3372105,1.128964,0.4829994,0.2435899,0.7527992,0.5115903,⋯,0.6651478,0.4964574,0.4129236,0.5533753,1.298231,0.4040472,1.828340,0.5204183,0.3277027,0.7256728
BCL2A1,0.9481724,0.7684480,0.96322671,0.5610849,1.0392105,1.649319,1.2241480,0.7758103,1.0028857,0.9682139,⋯,1.2424633,1.3308464,0.9141813,1.1113097,1.195891,1.2605281,1.418556,0.3751335,0.8535119,1.1056835


In [56]:
ID <- colnames(mean_df)

In [57]:
meta<-data.frame(ID)
head(meta, n = 3)

,ID
,<chr>
1,PB00001.01
2,PB00002.01
3,PB00003.01


In [58]:
# Extract the desired strings using regex
meta$pbmc_sample_id  <-  meta$ID
meta$pbmc_sample_id  <- str_replace_all(meta$pbmc_sample_id, "\\.", "-")
head(meta)

,ID,pbmc_sample_id
,<chr>,<chr>
1,PB00001.01,PB00001-01
2,PB00002.01,PB00002-01
3,PB00003.01,PB00003-01
4,PB00004.01,PB00004-01
5,PB00006.01,PB00006-01
6,PB00010.02,PB00010-02


In [59]:
# Left join the dataframes by the common column
merged_df <- left_join(meta, meta_data, by = "pbmc_sample_id")
head(merged_df)

,ID,pbmc_sample_id,X,Unnamed..0,lastUpdated,sample.id,sample.bridgingControl,sample.sampleKitGuid,sample.visitName,sample.visitDetails,⋯,subject.partnerCode,subject.race,subject.subjectGuid,cohort.cohortGuid,sample.diseaseStatesRecordedAtVisit,subject.covidVaxDose1.daysSinceFirstVisit,subject.covidVaxDose2.daysSinceFirstVisit,Covid_exclusion,subjectGuid,CMV
,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<lgl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,PB00001.01,PB00001-01,0,1,2024-05-10T17:12:44.959Z,f499ff83-e513-4d24-a10f-151348269fff,False,KT00001,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR,Caucasian,BR1001,BR1,NA,NA,NA,no,BR1001,Negative
2,PB00002.01,PB00002-01,1,2,2024-05-10T17:12:44.959Z,750e90a9-a296-4f0f-969f-60225c2bca17,False,KT00002,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR,Caucasian,BR1002,BR1,NA,440,461,no,BR1002,Negative
3,PB00003.01,PB00003-01,2,3,2024-05-10T17:12:44.959Z,2db6fb3f-e3f4-454b-891b-9b068541b51d,False,KT00003,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR,Caucasian,BR1003,BR1,NA,440,461,no,BR1003,Negative
4,PB00004.01,PB00004-01,3,4,2024-05-10T17:12:44.959Z,f04693c5-563c-4b5b-ae58-877d0d9ae2fe,False,KT00004,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR,Caucasian,BR1004,BR1,NA,543,563,no,BR1004,Negative
5,PB00006.01,PB00006-01,4,5,2024-05-10T17:12:44.959Z,eb5b3a3d-002e-40a6-aa19-aa0e6a7fff8f,False,KT00006,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR,Caucasian,BR1005,BR1,NA,451,492,no,BR1005,Negative
6,PB00010.02,PB00010-02,8,9,2024-05-10T17:12:44.959Z,f32ac061-0ede-4040-8d8c-35229ea5229c,False,KT00010,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR,Caucasian,BR1006,BR1,NA,510,538,no,BR1006,Negative


In [60]:
rownames(merged_df)<-merged_df$pbmc_sample_id
head(merged_df)

colnames(merged_df)
length(merged_df$pbmc_sample_id)

,ID,pbmc_sample_id,X,Unnamed..0,lastUpdated,sample.id,sample.bridgingControl,sample.sampleKitGuid,sample.visitName,sample.visitDetails,⋯,subject.partnerCode,subject.race,subject.subjectGuid,cohort.cohortGuid,sample.diseaseStatesRecordedAtVisit,subject.covidVaxDose1.daysSinceFirstVisit,subject.covidVaxDose2.daysSinceFirstVisit,Covid_exclusion,subjectGuid,CMV
,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<lgl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
PB00001-01,PB00001.01,PB00001-01,0,1,2024-05-10T17:12:44.959Z,f499ff83-e513-4d24-a10f-151348269fff,False,KT00001,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR,Caucasian,BR1001,BR1,NA,NA,NA,no,BR1001,Negative
PB00002-01,PB00002.01,PB00002-01,1,2,2024-05-10T17:12:44.959Z,750e90a9-a296-4f0f-969f-60225c2bca17,False,KT00002,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR,Caucasian,BR1002,BR1,NA,440,461,no,BR1002,Negative
PB00003-01,PB00003.01,PB00003-01,2,3,2024-05-10T17:12:44.959Z,2db6fb3f-e3f4-454b-891b-9b068541b51d,False,KT00003,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR,Caucasian,BR1003,BR1,NA,440,461,no,BR1003,Negative
PB00004-01,PB00004.01,PB00004-01,3,4,2024-05-10T17:12:44.959Z,f04693c5-563c-4b5b-ae58-877d0d9ae2fe,False,KT00004,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR,Caucasian,BR1004,BR1,NA,543,563,no,BR1004,Negative
PB00006-01,PB00006.01,PB00006-01,4,5,2024-05-10T17:12:44.959Z,eb5b3a3d-002e-40a6-aa19-aa0e6a7fff8f,False,KT00006,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR,Caucasian,BR1005,BR1,NA,451,492,no,BR1005,Negative
PB00010-02,PB00010.02,PB00010-02,8,9,2024-05-10T17:12:44.959Z,f32ac061-0ede-4040-8d8c-35229ea5229c,False,KT00010,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR,Caucasian,BR1006,BR1,NA,510,538,no,BR1006,Negative


[1] "ID"                                       
 [2] "pbmc_sample_id"                           
 [3] "X"                                        
 [4] "Unnamed..0"                               
 [5] "lastUpdated"                              
 [6] "sample.id"                                
 [7] "sample.bridgingControl"                   
 [8] "sample.sampleKitGuid"                     
 [9] "sample.visitName"                         
[10] "sample.visitDetails"                      
[11] "sample.drawDate"                          
[12] "sample.daysSinceFirstVisit"               
[13] "file.id"                                  
[14] "file.name"                                
[15] "file.batchID"                             
[16] "file.panel"                               
[17] "file.pool"                                
[18] "file.fileType"                            
[19] "file.majorVersion"                        
[20] "subject.id"                               
[21] "subject.biologicalSex"                    
[22] "subject.birthYear"                        
[23] "subject.ethnicity"                        
[24] "subject.partnerCode"                      
[25] "subject.race"                             
[26] "subject.subjectGuid"                      
[27] "cohort.cohortGuid"                        
[28] "sample.diseaseStatesRecordedAtVisit"      
[29] "subject.covidVaxDose1.daysSinceFirstVisit"
[30] "subject.covidVaxDose2.daysSinceFirstVisit"
[31] "Covid_exclusion"                          
[32] "subjectGuid"                              
[33] "CMV"

[1] 92

# Subsetting for each age and Sex individually

In [61]:
#subset for each sex and age
merged_df_v2 <- merged_df[merged_df$subject.biologicalSex == "Female" & merged_df$cohort.cohortGuid == "BR2", ]

#create a column called "Gene"
mean_df['Gene']= rownames(mean_df)

#filter the gene in count matrix
df <- mean_df %>% filter (Gene %in% c("PTGS2", "BCL2A1", "NLRP3", "OAZ1", "G0S2", "CMTM6", "RASGEF1B", "RIPK2", "RABL6",
                       "SGK1", "SELENOK", "METRNL", "MIDN", "DNAJC10", "BCL6", "PCBP2", "IL1B", "NR4A1", "FKBP5",
                       "SOCS3", "EREG", "TNF", "SHTN1", "WARS", "BAG1", "CDKN2D", "ATP1B3", "RPLP2", "ATP5PD",
                       "CCL3", "CCL4", "CXCL8"))
#remove column "gene" 
df <- df[,!(names(df) %in% c('Gene'))]

#transpose the data
df <- df %>% t() %>% as.data.frame()

#create a ID column for merging
df['ID']= rownames(df)

#merge the merged_df_v2 (with metadata) and the modified pseudobulk dataframe together
data <- inner_join(df,merged_df_v2,by=c('ID'))
#print(data)
#save the df
write_csv(data, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/CertPro_analysis/scripts/Figure_Notebooks/Intermediate_files/Mean_gene_expression/CSVs_mean_gene_exp_by_age_sex/IL1B_CD14mono_Female_BR2_mean_gene_exp.csv")

In [62]:
#subset for each sex and age
merged_df_v2 <- merged_df[merged_df$subject.biologicalSex == "Female" & merged_df$cohort.cohortGuid == "BR1", ]

#create a column called "Gene"
mean_df['Gene']= rownames(mean_df)

#filter the gene in count matrix
df <- mean_df %>% filter (Gene %in% c("PTGS2", "BCL2A1", "NLRP3", "OAZ1", "G0S2", "CMTM6", "RASGEF1B", "RIPK2", "RABL6",
                       "SGK1", "SELENOK", "METRNL", "MIDN", "DNAJC10", "BCL6", "PCBP2", "IL1B", "NR4A1", "FKBP5",
                       "SOCS3", "EREG", "TNF", "SHTN1", "WARS", "BAG1", "CDKN2D", "ATP1B3", "RPLP2", "ATP5PD",
                       "CCL3", "CCL4", "CXCL8"))
#remove column "gene" 
df <- df[,!(names(df) %in% c('Gene'))]

#transpose the data
df <- df %>% t() %>% as.data.frame()

#create a ID column for merging
df['ID']= rownames(df)

#merge the merged_df_v2 (with metadata) and the modified pseudobulk dataframe together
data <- inner_join(df,merged_df_v2,by=c('ID'))

#save the df
write_csv(data, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/CertPro_analysis/scripts/Figure_Notebooks/Intermediate_files/Mean_gene_expression/CSVs_mean_gene_exp_by_age_sex/IL1B_CD14mono_Female_BR1_mean_gene_exp.csv")

In [63]:
#subset for each sex and age
merged_df_v2 <- merged_df[merged_df$subject.biologicalSex == "Male" & merged_df$cohort.cohortGuid == "BR2", ]

#create a column called "Gene"
mean_df['Gene']= rownames(mean_df)

#filter the gene in count matrix
df <- mean_df %>% filter (Gene %in% c("PTGS2", "BCL2A1", "NLRP3", "OAZ1", "G0S2", "CMTM6", "RASGEF1B", "RIPK2", "RABL6",
                       "SGK1", "SELENOK", "METRNL", "MIDN", "DNAJC10", "BCL6", "PCBP2", "IL1B", "NR4A1", "FKBP5",
                       "SOCS3", "EREG", "TNF", "SHTN1", "WARS", "BAG1", "CDKN2D", "ATP1B3", "RPLP2", "ATP5PD",
                       "CCL3", "CCL4", "CXCL8"))
#remove column "gene" 
df <- df[,!(names(df) %in% c('Gene'))]

#transpose the data
df <- df %>% t() %>% as.data.frame()

#create a ID column for merging
df['ID']= rownames(df)

#merge the merged_df_v2 (with metadata) and the modified pseudobulk dataframe together
data <- inner_join(df,merged_df_v2,by=c('ID'))

#save the df
write_csv(data, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/CertPro_analysis/scripts/Figure_Notebooks/Intermediate_files/Mean_gene_expression/CSVs_mean_gene_exp_by_age_sex/IL1B_CD14mono_Male_BR2_mean_gene_exp.csv")

In [64]:
#subset for each sex and age
merged_df_v2 <- merged_df[merged_df$subject.biologicalSex == "Male" & merged_df$cohort.cohortGuid == "BR1", ]

#create a column called "Gene"
mean_df['Gene']= rownames(mean_df)

#filter the gene in count matrix
df <- mean_df %>% filter (Gene %in% c("PTGS2", "BCL2A1", "NLRP3", "OAZ1", "G0S2", "CMTM6", "RASGEF1B", "RIPK2", "RABL6",
                       "SGK1", "SELENOK", "METRNL", "MIDN", "DNAJC10", "BCL6", "PCBP2", "IL1B", "NR4A1", "FKBP5",
                       "SOCS3", "EREG", "TNF", "SHTN1", "WARS", "BAG1", "CDKN2D", "ATP1B3", "RPLP2", "ATP5PD",
                       "CCL3", "CCL4", "CXCL8"))
#remove column "gene" 
df <- df[,!(names(df) %in% c('Gene'))]

#transpose the data
df <- df %>% t() %>% as.data.frame()

#create a ID column for merging
df['ID']= rownames(df)

#merge the merged_df_v2 (with metadata) and the modified pseudobulk dataframe together
data <- inner_join(df,merged_df_v2,by=c('ID'))

#save the df
write_csv(data, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/CertPro_analysis/scripts/Figure_Notebooks/Intermediate_files/Mean_gene_expression/CSVs_mean_gene_exp_by_age_sex/IL1B_CD14mono_Male_BR1_mean_gene_exp.csv")